# Spam Classification with GPT-2
Fine-tuning GPT-2 on the SMS Spam Collection dataset for binary classification.


In [ ]:
!pip install torchmetrics

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

In [ ]:
!unzip -o smsspamcollection.zip

In [6]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [7]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2025-04-23 22:43:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.3’

smsspamcollection.z     [   <=>              ] 198.65K   308KB/s    in 0.6s    

2025-04-23 22:43:45 (308 KB/s) - ‘smsspamcollection.zip.3’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [8]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
text = df.text.values
labels = df.label.values

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = 'left'

In [11]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

Yunny... I'm goin to be late

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ Y        │          56 │
├──────────┼─────────────┤
│ unny     │       16948 │
├──────────┼─────────────┤
│ ...      │         986 │
├──────────┼─────────────┤
│ ĠI       │         314 │
├──────────┼─────────────┤
│ 'm       │        1101 │
├──────────┼─────────────┤
│ Ġgo      │         467 │
├──────────┼─────────────┤
│ in       │         259 │
├──────────┼─────────────┤
│ Ġto      │         284 │
├──────────┼─────────────┤
│ Ġbe      │         307 │
├──────────┼─────────────┤
│ Ġlate    │        2739 │
├──────────┼─────────────┤
│ Ċ        │         198 │
╘══════════╧═════════════╛


In [12]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):

  return tokenizer.encode_plus(
      input_text,
      max_length=32,
      padding='max_length',
      truncation=True,
      return_tensors='pt',
      return_attention_mask=True
  )


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [13]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', 'Just', 'Ġnow', 'Ġsaw', 'Ġyour', 'Ġmessage', '.', 'it', 'Ġk', 'Ġda', ':', ')', 'Ċ']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|

In [14]:
val_ratio = 0.2
# Batch size (chosen 16 here for balance between speed and memory)
batch_size = 16

# Use train_test_split
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size=val_ratio,
    random_state=42,
    stratify=labels.numpy()  # ensure same label distribution
)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], labels[train_idx])

val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)

validation_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last=False)

### Load specific versions of the model

In [15]:
# Load GPT-2 for sequence classification with 2 labels

config = GPT2Config.from_pretrained(
    'gpt2',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = tokenizer.eos_token_id

# Learning rate for AdamW optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [16]:
# Select device (GPU if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
model = model.to(device)

epochs = 2

In [18]:
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = True # I set to 'False' in my second experiment to use GPT-2 as a frozen feature extractor

# Turn off gradients using the above
if not fine_tune:
    for param in model.transformer.parameters():
        param.requires_grad = False

total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_parameters}")
assert (fine_tune and total_parameters == 124441344) or (not fine_tune and total_parameters < 124441344)

Total trainable parameters: 124441344


### Train the model

In [19]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = Accuracy(task="binary").to(device)
recall = Recall(task="binary").to(device)
precision = Precision(task="binary").to(device)
auroc = AUROC(task="binary").to(device)

In [20]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = train_output.loss
        # Backward pass
        loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_labels.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        # Unpack the batch
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
          # Forward pass
            eval_output = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

        # Calculate validation metrics
        labels = b_labels
        logits = eval_output.logits
        predicted_labels = torch.argmax(logits, dim=1)
        probs = torch.softmax(logits, dim=1)
        preds_prob = probs[:, 1]

        val_accuracy.append(accuracy(predicted_labels, labels).item())
        val_recall.append(recall(predicted_labels, labels).item())
        val_precision.append(precision(predicted_labels, labels).item())
        val_auroc.append(auroc(preds_prob, labels).item())

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Epoch:  50%|█████     | 1/2 [00:12<00:12, 12.92s/it]


	 - Train loss: 0.1100
	 - Validation Accuracy: 0.9884
	 - Validation Precision: 0.8714
	 - Validation Recall: 0.8186
	 - Validation AUROC: 0.9075



Epoch: 100%|██████████| 2/2 [00:25<00:00, 12.71s/it]


	 - Train loss: 0.0428
	 - Validation Accuracy: 0.9902
	 - Validation Precision: 0.8810
	 - Validation Recall: 0.8738
	 - Validation AUROC: 0.9095



### Test on a specific sentence, see the outcome

In [21]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


## Experiment: Fine-Tuning vs Frozen GPT-2

We compared two different training strategies for GPT-2 on the SMS Spam dataset:

- **Fine-Tuned GPT-2 (Full Model Training)**  
  All transformer layers and the classification head were trained end-to-end.

- **Frozen GPT-2 (Feature Extractor)**  
  The GPT-2 transformer layers were kept frozen, and only the classification head was trained.

---

### Results

**Fine-Tuned GPT-2 (Epoch 2)**  
- Validation Accuracy: **99.02%**  
- Precision: **88.10%**  
- Recall: **87.38%**  
- AUROC: **90.95%**

**Frozen GPT-2 (Epoch 2)**  
- Validation Accuracy: **91.44%**  
- Precision: **62.26%**  
- Recall: **52.02%**  
- AUROC: **83.69%**

---

### Performance Gap Summary

| Metric      | Fine-Tuned GPT-2 | Frozen GPT-2 | Gap     |
|-------------|------------------|--------------|---------|
| Accuracy    | 99.02%           | 91.44%       | -7.58%  |
| Precision   | 88.10%           | 62.26%       | -25.84% |
| Recall      | 87.38%           | 52.02%       | -35.36% |
| AUROC       | 90.95%           | 83.69%       | -7.26%  |

---

### Conclusion

Fine-tuning the entire GPT-2 model significantly boosts all evaluation metrics compared to using it as a static feature extractor.  
The gains are most notable in **recall** and **precision**, which are critical for spam detection tasks.  
This highlights the importance of adapting pretrained language models through end-to-end fine-tuning for task-specific performance.
